# Задание 1

Примените PCA к данным MNIST.

Попробуйте со скалированием и без. Где надо меньше компонент при требовании 90% объясняемой дисперсии? (код визуализации ниже)

Обучите knn классификатор на исходных данных и на выбранных компонентах PCA (со скалированием и без). Сравните тестовый скор по метрике accuracy.

In [ ]:
from sklearn import datasets 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier


%matplotlib inline

In [ ]:
digits = datasets.load_digits()
X, y = digits['data'], digits['target']

In [ ]:
plt.figure(figsize=(16, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X[i,:].reshape([8,8]));

Сначала визуализируем без скалирования и с ним:

In [ ]:
pca = PCA(n_components=2)

#ваш код здесь
X_reduced = pca.fit_transform(X)

print('Projecting %d-dimensional data to 2D' % X.shape[1])

plt.figure(figsize=(12,10))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y, 
            edgecolor='none', alpha=0.7, s=40,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.colorbar()
plt.title('MNIST. PCA projection')

In [ ]:
# если стандартизировать
df = digits['data']

ss = StandardScaler()
df = ss.fit_transform(df)

pca = PCA(n_components=2)
df_reduced = pca.fit_transform(df)

print('Projecting %d-dimensional data to 2D' % df.shape[1])

plt.figure(figsize=(12,10))
plt.scatter(df_reduced[:, 0], df_reduced[:, 1], c=y, 
            edgecolor='none', alpha=0.7, s=40,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.colorbar()
plt.title('MNIST. PCA projection')

### Пример визуализации выбора кол-ва компонент

In [ ]:
# Без скалирования
def find_comp(list_of_cumsum):
    for i in range(len(list_of_cumsum)):
        if list_of_cumsum[i] > 0.9:
            return i
        
pca = PCA().fit(X) #ваши данные

plt.figure(figsize=(10,7))
plt.plot(np.cumsum(pca.explained_variance_ratio_), color='k', lw=2)
plt.xlabel('Number of components')
plt.ylabel('Total explained variance')
plt.title('There are {} components'. format(find_comp(np.cumsum(pca.explained_variance_ratio_))))
plt.xlim(0, 63)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.axvline(find_comp(np.cumsum(pca.explained_variance_ratio_)), c='b')
plt.axhline(0.9, c='r')
plt.show();

In [ ]:
# Со скалированием
def find_comp(list_of_cumsum):
    for i in range(len(list_of_cumsum)):
        if list_of_cumsum[i] > 0.9:
            return i
        
pca = PCA().fit(df) #ваши данные

plt.figure(figsize=(10,7))
plt.plot(np.cumsum(pca.explained_variance_ratio_), color='k', lw=2)
plt.xlabel('Number of components')
plt.ylabel('Total explained variance')
plt.title('There are {} components'. format(find_comp(np.cumsum(pca.explained_variance_ratio_))))
plt.xlim(0, 63)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.axvline(find_comp(np.cumsum(pca.explained_variance_ratio_)), c='b')
plt.axhline(0.9, c='r')
plt.show();

Посчитаем модели:

In [ ]:
v_metric = []
n=1

for i in range(1, 15):
    kf = KFold(n_splits=10, shuffle=True, random_state=777)   
    for tr_ind, val_ind in kf.split(X):
#         print('Start {} fold'.format(n))

        val = X[val_ind]
        ttt = X[tr_ind] 

        clf = KNeighborsClassifier(n_neighbors=i)

        clf.fit(ttt, y[tr_ind]) 

        model_pred_valid = clf.predict(val)

        y_valid = y[val_ind]
        valid_metric = accuracy_score(y_valid, model_pred_valid)
        v_metric.append(valid_metric)

        n+=1

    print('n_neighbors', i, 'crossval score:', np.mean(v_metric), 'std', np.std(v_metric))
    print('---------------------------------------')

In [ ]:
pca = PCA(n_components=20).fit_transform(X)

v_metric = []
n=1

for i in range(1, 15):
    kf = KFold(n_splits=10, shuffle=True, random_state=777)   
    for tr_ind, val_ind in kf.split(pca):
#         print('Start {} fold'.format(n))

        val = X[val_ind]
        ttt = X[tr_ind] 

        clf = KNeighborsClassifier(n_neighbors=i)

        clf.fit(ttt, y[tr_ind]) 

        model_pred_valid = clf.predict(val)

        y_valid = y[val_ind]
        valid_metric = accuracy_score(y_valid, model_pred_valid)
        v_metric.append(valid_metric)

        n+=1

    print('n_neighbors', i, 'crossval score:', np.mean(v_metric), 'std', np.std(v_metric))
    print('---------------------------------------')

In [ ]:
pca = PCA(n_components=30).fit_transform(df)

v_metric = []
n=1

for i in range(1, 15):
    kf = KFold(n_splits=10, shuffle=True, random_state=777)   
    for tr_ind, val_ind in kf.split(pca):
#         print('Start {} fold'.format(n))

        val = X[val_ind]
        ttt = X[tr_ind] 

        clf = KNeighborsClassifier(n_neighbors=i)

        clf.fit(ttt, y[tr_ind]) 

        model_pred_valid = clf.predict(val)

        y_valid = y[val_ind]
        valid_metric = accuracy_score(y_valid, model_pred_valid)
        v_metric.append(valid_metric)

        n+=1

    print('n_neighbors', i, 'crossval score:', np.mean(v_metric), 'std', np.std(v_metric))
    print('---------------------------------------')

У нас такие данные, что результаты работы моделей получились идентичные.